In [8]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [9]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
### Load NER model
model_ner = spacy.load('./output/model-best/')
model_ner

In [12]:
# Load Image
image = cv2.imread('./test_templates/Alendemeltest.png')

#cv2.imshow('businesscard',image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [13]:
# extract data using Pytesseract 
tessData = pytesseract.image_to_data(image)

In [14]:
# convert into dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)
df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,793,1123,-1,
1,2,1,1,0,0,0,270,92,251,30,-1,
2,3,1,1,1,0,0,270,92,251,30,-1,
3,4,1,1,1,1,0,270,92,251,30,-1,
4,5,1,1,1,1,1,270,92,83,30,96.573158,alan
...,...,...,...,...,...,...,...,...,...,...,...,...
221,4,1,12,1,2,0,367,997,58,9,-1,
222,5,1,12,1,2,1,367,997,58,9,76.755676,4666331
223,4,1,12,1,3,0,274,1012,246,13,-1,
224,5,1,12,1,3,1,274,1013,54,9,93.223030,e-mail


In [15]:
# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)

alan de maid works order to srarter appliances ltd date 23/05/2024 m stedeford ‘order bros-16009551 386-137 high street shracombe ‘ordered by jeri davison bromley pmc bg4 sez con.fax access tenant key return keys please execute the appliance - repairs at oionods24 property 000004324 11 hilda vale close ‘orpington bre tah contact ms stephanie susan sawyer 07742 677619 cooker in white - hotpoint indesit -£649vat delivery, installation and removal - £125 vat. stoptap please contact the office below with a quotation prior to carrying out any works. required by as this works order has been placed on behalf of the landlord, your invoice must be addressed directly to the landlord mr john worton jo bromley adm and alan de maid bromley 18 east street, bromey ent br 1qe l_ _ please email / send all invoices to the office address below office address 19 east street, bromley, kent, bri 1qe phone 02084667172 fax 0208 4666331 e-mail lettings@alandemaid.co.uk


In [16]:
# get prediction from NER model
doc = model_ner(content)

In [17]:
from spacy import displacy

In [18]:
#displacy.serve(doc,style='ent')

In [19]:
displacy.render(doc,style='ent')

In [20]:
### Tagging
docjson = doc.to_json()
docjson.keys()


dict_keys(['text', 'ents', 'tokens'])

In [21]:
doc_text = docjson['text']

In [22]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,alan
1,1,5,7,de
2,2,8,12,maid
3,3,13,18,works
4,4,19,24,order
5,5,25,27,to
6,6,28,35,srarter
7,7,36,46,appliances
8,8,47,50,ltd
9,9,51,55,date


In [23]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)



,id,start,end,token,label
0,0,0,4,alan,I-PAYMENTCOMPANYNAME
1,1,5,7,de,I-PAYMENTCOMPANYNAME
2,2,8,12,maid,I-PAYMENTCOMPANYNAME
3,3,13,18,works,I-PAYMENTBYERADDRESS
4,4,19,24,order,O
5,5,25,27,to,O
6,6,28,35,srarter,O
7,7,36,46,appliances,O
8,8,47,50,ltd,O
9,9,51,55,date,O


In [24]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

# inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
146,5,1,12,1,1,7,369,981,34,11,91.937225,"kent,",882,877,kent,O
147,5,1,12,1,1,8,409,981,25,9,41.557808,bri,886,883,bri,O
148,5,1,12,1,1,9,440,981,25,11,71.149490,1qe,890,887,1qe,O
149,5,1,12,1,1,10,475,981,53,9,85.978706,phone,896,891,phone,B-PAYMENTBYERPHONE
150,5,1,12,1,1,11,538,981,101,9,74.419983,02084667172,908,897,02084667172,I-PAYMENTBYERPHONE
151,5,1,12,1,1,12,652,981,32,9,94.912079,fax,912,909,fax,B-PAYMENTBYERFAX
152,5,1,12,1,1,13,694,981,33,9,95.495956,0208,917,913,0208,I-PAYMENTBYERFAX
153,5,1,12,1,2,1,367,997,58,9,76.755676,4666331,925,918,4666331,I-PAYMENTBYERFAX
154,5,1,12,1,3,1,274,1013,54,9,93.223030,e-mail,932,926,e,B-PAYMENTBYEREMAIL
155,5,1,12,1,3,2,339,1012,181,13,66.468765,lettings@alandemaid.co.uk,958,933,lettings@alandemaid.co.uk,I-PAYMENTBYEREMAIL


In [25]:
### Bounding Box

bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,1,1,1,1,270,92,83,30,96.573158,alan,4,0,alan,PAYMENTCOMPANYNAME
1,5,1,1,1,1,2,370,92,45,30,96.573158,de,7,5,de,PAYMENTCOMPANYNAME
2,5,1,1,1,1,3,432,92,89,30,96.904709,maid,12,8,maid,PAYMENTCOMPANYNAME
3,5,1,4,1,1,1,345,176,51,9,95.187477,works,18,13,works,PAYMENTBYERADDRESS
10,5,1,4,1,2,7,586,225,67,12,92.514702,23/05/2024,66,56,23/05/2024,ORDERDATE


In [27]:
# group the label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

img_tagging


,left,right,top,bottom,label,token
group,,,,,,
1,270,521,92,122,[PAYMENTCOMPANYNAME],alan de maid
2,345,396,176,185,[PAYMENTBYERADDRESS],works
3,586,653,225,237,[ORDERDATE],23/05/2024
4,588,680,236,265,[PAYMENTPONUMBER],bros-16009551
5,132,386,374,385,[FAULTDETAIL],execute appliance - repairs
6,140,598,397,411,[INSTRUCTIONNOTES],at oionods24 property
7,609,673,399,408,[FAULTDETAIL],000004324
8,185,276,409,444,[INSTRUCTIONNOTES],11 hilda vale close ‘ bre
9,168,183,566,575,[SHIPPINGNAME],ms


In [30]:
img_bb = image.copy()
for l, r, t, b, label, token in img_tagging.values:
    cv2.rectangle(img_bb, (l, t), (r, b), (0, 255, 0), 2)
    
    # Ensure label is a string
    cv2.putText(img_bb, str(label), (l, t), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 255), 2)

cv2.imshow('Bounding Box workorder', img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [46]:
import re
from datetime import datetime

def parser(text, label):
    text = text.strip()  # General clean-up for all labels
    
    if label in ['PAYMENTBYERPHONE', 'SHIPPINGPHONE']:
        # Standardize phone numbers by keeping only digits
        text = re.sub(r'\D', '', text)
        
    elif label in ['[PAYMENTBYEREMAIL]']:
        # Standardize email (keep alphanumeric characters, @, ., _, and -)
        text = text.lower()
        allowed_chars = '@_.\-'
        text = re.sub(rf'[^A-Za-z0-9{allowed_chars}]', '', text)
        
    elif label == 'WEB':
        # Standardize URLs by allowing common URL characters
        text = text.lower()
        allowed_chars = ':/.%#\-'
        text = re.sub(rf'[^A-Za-z0-9{allowed_chars}]', '', text)
        
    elif label in ['PAYMENTBYERNAME', 'SHIPPINGNAME', 'PAYMENTCOMPANYNAME']:
        # Standardize names (title case, allow alphabet and spaces)
        text = re.sub(r'[^a-zA-Z ]', '', text).title()
        
    elif label in ['FAULTDETAIL', 'INSTRUCTIONNOTES']:
        # Allow alphanumeric, spaces, and basic punctuation
        text = re.sub(r'[^a-zA-Z0-9.,!? ]', '', text).strip() 
         
    elif label in ['PAYMENTCOMPANYNAME', 'PAYMENTBYERADDRESS', 'SHIPPINGSTREET']:
        # Standardize company names and addresses (allow alphanumeric and spaces)
        text = re.sub(r'[^a-zA-Z0-9 ]', '', text).title()
        
    elif label in ['REQUIREDDATE', 'ORDERDATE']:
        # Attempt to parse and standardize dates
        try:
            text = datetime.strptime(text, '%d/%m/%Y').strftime('%Y-%m-%d')
        except ValueError:
            pass  # Keep original text if parsing fails
    else:
        # General clean up for other labels (remove extra spaces)
        text = text.strip()
    return text


In [38]:
# Example data
#test_data = [
#    ("0208 4666331", "[PAYMENTBYERPHONE]"),
#    ("lettings@alandemaid.co.uk", "[PAYMENTBYEREMAIL]"),
#    ("Alan de Maid", "[PAYMENTCOMPANYNAME]"),
#    ("23/05/2024", "[REQUIREDDATE]"),
#    ("Bromley ADM and Alan de Maid", "[PAYMENTCOMPANYNAME]"),
#]

#for text, label in test_data:
#    cleaned_text = parser(text, label)
#    print(f"Label: {label}, Parsed: {cleaned_text}")
#parser('03#45@$423432432 srfdsfds','[PAYMENTBYERPHONE]')

In [47]:
# Assuming `info_array` is your input array with tokens and labels
info_array = dataframe_info[['token', 'label']].values  # Make sure `dataframe_info` exists

entities = {
    'PAYMENTBYERPHONE': [],
    'PAYMENTBYERFAX': [],
    'PAYMENTBYEREMAIL': [],
    'PAYMENTCOMPANYNAME': [],
    'PAYMENTBYERADDRESS': [],
    'PAYMENTBYERNAME':[],
    'ORDERDATE': [],
    'ACCESSKEY':[], 
    'FAULTDETAIL': [],
    'INSTRUCTIONNOTES': [],
    'SHIPPINGNAME': [],
    'SHIPPINGPHONE': [],
    'SHIPPINGSTREET':[],
    'PAYMENTBILLINGNAME': [],
    'PAYMENTPONUMBER': [],
}

previous_label_tag = 'O'

# Parser function (assuming it is defined previously)
def parser(text, label):
    # Add your custom parsing logic here
    return text.strip()

# Extract entities
for token, label in info_array:
    if label == 'O':  # Ignore tokens without entity labels
        continue
    bio_tag, label_tag = label.split('-')
    parsed_token = parser(token, label_tag)

    if bio_tag == 'B' or previous_label_tag != label_tag:
        # Start a new entity
        entities[label_tag].append(parsed_token)
    elif bio_tag == 'I' and previous_label_tag == label_tag:
        # Continue the current entity
        if label_tag in ['PAYMENTCOMPANYNAME', 'SHIPPINGNAME', 'PAYMENTBILLINGNAME']:
            entities[label_tag][-1] += f" {parsed_token}"
        else:
            entities[label_tag][-1] += parsed_token

    previous_label_tag = label_tag

print(entities)


{'PAYMENTBYERPHONE': ['phone02084667172'], 'PAYMENTBYERFAX': ['fax02084666331'], 'PAYMENTBYEREMAIL': ['elettings@alandemaid.co.uk'], 'PAYMENTCOMPANYNAME': ['alan de maid', 'bromley adm and alan de maid bromley'], 'PAYMENTBYERADDRESS': ['works', '18east1qel'], 'PAYMENTBYERNAME': [], 'ORDERDATE': ['23/05/2024'], 'ACCESSKEY': [], 'FAULTDETAIL': ['executeappliance-repairs', '000004324'], 'INSTRUCTIONNOTES': ['atoionods24property', '11hildavaleclose‘bre', '677619cookerinwhite-hotpointindesit-£649vatdeliveryinstallationandremoval-£vat'], 'SHIPPINGNAME': ['ms', 'susan sawyer'], 'SHIPPINGPHONE': ['07742'], 'SHIPPINGSTREET': [], 'PAYMENTBILLINGNAME': ['stephanie', 'mr john worton jo'], 'PAYMENTPONUMBER': ['bros-16009551']}
